In [1]:
from sumTree import Memory
from DQN import DQNAgent
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import datetime
import tensorflow as tf
import numpy as np
import gymnasium as gym

In [ ]:
# Create TensorBoard callback
log_dir = "logs/dqn_lunar" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# alada logs/dqn or Log/ddqn
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


checkpoint_path = "checkpoints/dqn_lunar_checkpoint_{epoch:02d}.weights.h5"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,  # Only save weights, not the full model
    save_best_only=False,
    save_freq='epoch',       # Save every epoch (you can change this)
    verbose=1
)

model_save_path = "saved_models/dqn_model_lunar"


# Lists to store metrics for each episode
episode_rewards = []
episode_epsilons = []

# to write the tensorboard logs
writer = tf.summary.create_file_writer(log_dir)


env = gym.make('LunarLander-v2', render_mode='human')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

agent = DQNAgent(state_size, action_size)
scores = []
EPISODES = 2000
avg_window = 100

save_interval = 100

for e in range(EPISODES):
    state, _ = env.reset(seed=42)
    # print(f'state : {state}')
    state = np.reshape(state, [1, state_size])
    done = False
    time = 0
    total_reward = 0

    while not done:
        time += 1
        action = agent.act(state)
        next_state, reward, done, _, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.memorize(state, action, reward, next_state, done)
        state = next_state

        total_reward += reward

        if done:
            scores.append(total_reward)
            if e % 100 == 0 and e > 1:
                print("episode: {}/{}, Score Mean: {} / Median: {} ".format(e, EPISODES, int(np.mean(scores)), int(np.median(scores))))
                print("Beta {:.5f} / Eps: {:.5f}".format(agent.memory.beta, agent.epsilon))
            # scores.append(time)
    if agent.memory.tree.n_entries > 1000:
        agent.replay()

    # Save manually if desired or use replay
    if e % save_interval == 0:
        agent.model.save(f"{model_save_path}_episode_{e}.h5")

    # Use the callback for automated saving
    if e == 0:  # First save
        checkpoint_callback.set_model(agent.model)

    with writer.as_default():
        tf.summary.scalar('Total Reward', total_reward, step=e)
        tf.summary.scalar('Epsilon', agent.epsilon, step=e)

        # Compute and log the average reward over the last 100 episodes
        if len(scores) >= avg_window:
            avg_reward = np.mean(scores[-avg_window:])
            tf.summary.scalar('Average Reward (last 100 episodes)', avg_reward, step=e)

env.close()

c:\Users\jahin\anaconda3\envs\gymenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━